<h1> Identifying spam in popular YouTube videos comments </h1>
<p1>TODO: Short Description of the data and the approach.</p1>
<p2>TODO: Description of the code in all cells </p2>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
import zipfile
COMMENTS_FT = 'comments_words'
ZIP_FILE = 'data/YouTube-Spam-Collection-v1.zip'
DIR_TO_EXTRACT = 'data/'
EMBED_DIMENSION = 50

#Extract the Data
zip_ref = zipfile.ZipFile(ZIP_FILE, 'r')
zip_ref.extractall(DIR_TO_EXTRACT)
zip_ref.close()

/Users/i346047/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/i346047/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
comments_df_list = []
comments_file = ['data/Youtube01-Psy.csv','data/Youtube02-KatyPerry.csv','data/Youtube03-LMFAO.csv',
                'data/Youtube04-Eminem.csv','data/Youtube05-Shakira.csv']
for f in comments_file:
    df = pd.read_csv(f,header=0)
    comments_df_list.append(df)
comments_df = pd.concat(comments_df_list)
comments_df = comments_df.sample(frac=1.0)
print(comments_df.shape)
comments_df.head(5)

(1956, 5)


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
326,z12pyzwiourgc1t2f22wzj5xuoeqvxdp104,Ruben Aviles,2015-01-22T02:08:41.056000,Check out this video on YouTube:﻿,1
166,z13mebgjooulgzdkt23kzvlqjyrzx32n0,amine moha,2014-11-06T18:11:37,please like : http://www.bubblews.com/news/927...,1
96,z121ft552pzpzhiab04cffmwitfii3lr5v40k,Crooked Gaming,2014-11-04T00:00:57,I made a gaming channel (Unique right?) :L Ang...,1
244,z13jddxactekydkim04cfvzijxiygbvgkh4,Patricia Reyes,2014-10-23T05:24:14,I love the song roar it make me think am fill ...,0
248,z12vcv5xuqq4gvpgu04ch5ah3vyqwzphryc0k,kissmehk,2015-05-23T19:37:52.240000,:)﻿,0


In [3]:
comments_df.columns

Index(['COMMENT_ID', 'AUTHOR', 'DATE', 'CONTENT', 'CLASS'], dtype='object')

In [4]:
average_comments_size = int(sum([len(c) for c in comments_df.CONTENT])/comments_df.shape[0])
print(average_comments_size)

94


In [5]:
voc_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(average_comments_size)
X_transform = voc_processor.fit_transform(comments_df.CONTENT)
X_transform = np.array(list(X_transform))
y = comments_df.CLASS.values
X_tr, X_te, y_train, y_test = model_selection.train_test_split(X_transform, 
                                    y, test_size=0.2, random_state=42)
n_words = len(voc_processor.vocabulary_)

In [6]:
voc_dict = voc_processor.vocabulary_._mapping
sorted_vocab = sorted(voc_dict.items(), key = lambda x : x[1])
f = open('/tmp/meta.tsv', 'w')
for val in sorted_vocab:
    f.write(str(val[0]) + "\n")
f.close()

In [7]:
def get_estimator_spec(inp_lgits, out_lb, trn_prd_m):
    preds_cls = tf.argmax(inp_lgits, 1)
    if trn_prd_m == tf.estimator.ModeKeys.PREDICT:
        preds_vals = {'prd_class': preds_cls, 'prd_pr': tf.nn.softmax(inp_lgits)}
        return tf.estimator.EstimatorSpec(
        mode=trn_prd_m,
        predictions=preds_vals)
    tr_l = tf.losses.sparse_softmax_cross_entropy(labels=out_lb, logits=inp_lgits)
    if trn_prd_m == tf.estimator.ModeKeys.TRAIN:
        adm_opt = tf.train.AdamOptimizer(learning_rate=0.01)
        tr_op = adm_opt.minimize(tr_l, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(trn_prd_m, loss=tr_l, train_op=tr_op)
    eval_metric_ops = {'prd_acc': tf.metrics.accuracy(labels=out_lb, predictions=preds_cls)}
    return tf.estimator.EstimatorSpec(trn_prd_m, loss=tr_l, train_op=tr_op)

In [8]:
def rnn_model_fn(features, labels, mode):
    comments_wd_vec = tf.contrib.layers.embed_sequence(
      features[COMMENTS_FT], vocab_size=n_words, embed_dim=EMBED_DIMENSION)
    comments_word_list = tf.unstack(comments_wd_vec, axis=1)
    
    rnn_cell = tf.nn.rnn_cell.GRUCell(average_comments_size)
    
    _, comments_encoding = tf.nn.static_rnn(rnn_cell, comments_word_list, dtype=tf.float32)
    
    logits = tf.layers.dense(inputs=comments_encoding, units=2, activation=None)
    
    return get_estimator_spec(logits, labels, mode)

In [9]:
run_config = tf.contrib.learn.RunConfig()
run_config = run_config.replace(model_dir='/tmp/models/',save_summary_steps=10,log_step_count_steps=10)
classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,config=run_config)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114733c50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_secs': 600, '_log_step_count_steps': 10, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/models/'}


In [10]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={COMMENTS_FT: X_tr},
      y=y_train,
      batch_size=128,
      num_epochs=None,
      shuffle=True)
classifier.train(input_fn=train_input_fn, steps=200)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/model.ckpt.
INFO:tensorflow:loss = 0.6912304, step = 1
INFO:tensorflow:global_step/sec: 3.09311
INFO:tensorflow:global_step/sec: 11.8687
INFO:tensorflow:global_step/sec: 11.8699
INFO:tensorflow:global_step/sec: 11.8364
INFO:tensorflow:global_step/sec: 11.9659
INFO:tensorflow:global_step/sec: 11.551
INFO:tensorflow:global_step/sec: 11.3143
INFO:tensorflow:global_step/sec: 11.5508
INFO:tensorflow:global_step/sec: 11.9426
INFO:tensorflow:global_step/sec: 11.9586
INFO:tensorflow:loss = 0.0039109336, step = 101 (10.889 sec)
INFO:tensorflow:global_step/sec: 11.8764
INFO:tensorflow:global_step/sec: 11.4342
INFO:tensorflow:global_step/sec: 11.9519
INFO:tensorflow:global_step/sec: 11.9224
INFO:tensorflow:global_step/sec: 11.8756
INFO:tensorflow:global_step/sec: 11.3042
INFO:tensorflow:global_step/sec: 11.4061
INFO:tensorflow:global_step/sec: 11.5409
INFO:tensorflow:global_step/sec: 11.9171
INFO

In [11]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={COMMENTS_FT: X_te},
      y=y_test,
      num_epochs=1,
      shuffle=False)
preds = classifier.predict(input_fn=test_input_fn)
y_prd = np.array(list(p['prd_class'] for p in preds))
y_prd = y_prd.reshape(np.array(y_test).shape)

acc = metrics.accuracy_score(y_test, y_prd)
print('Accuracy: {0:f}'.format(acc))

INFO:tensorflow:Restoring parameters from /tmp/models/model.ckpt-200
Accuracy: 0.956633
